In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ipywidgets import Output, HBox, VBox
from IPython import display
import pandas as pd
import ipyanchorviz as av

In [ ]:
av.__version__

# Helper Functions

In [ ]:
def testing_dash(viz: av.AnchorViz):
    anchor_display = Output()
    data_display = Output()
    lassoed_data_display = Output()
    selected_anchor_display = Output()
    selected_point_display = Output()
    def anchorchanges(change):
        with anchor_display:
            display.clear_output(wait=True)
            display.display(pd.DataFrame(viz.anchors))
    def datachanges(change):
        with data_display:
            display.clear_output(wait=True)
            display.display(pd.DataFrame(viz.data))
    def lassochanges(change):
        data = pd.DataFrame(viz.data)
        with lassoed_data_display:
            display.clear_output(wait=True)
            if "id" in data.columns:
                display.display(data[data.id.isin(viz.lassoedPointIDs)])
    def selectedanchorchanges(change):
        with selected_anchor_display:
            display.clear_output(wait=True)
            print("selected anchor ID:", viz.selectedAnchorID)
    def selecteddatachanges(change):
        with selected_point_display:
            display.clear_output(wait=True)
            print("selected data point ID:", viz.selectedDataPointID)
    viz.observe(anchorchanges, names="anchors")
    viz.observe(datachanges, names="data")
    viz.observe(lassochanges, names="lassoedPointIDs")
    viz.observe(selectedanchorchanges, names="selectedAnchorID")
    viz.observe(selecteddatachanges, names="selectedDataPointID")

    anchorchanges(None)
    datachanges(None)
    lassochanges(None)
    
    return HBox([viz, VBox([anchor_display, data_display]), VBox([lassoed_data_display, selected_anchor_display, selected_point_display])])

In [ ]:
def simple_testing_data():
    anchors = [
        {"id": "0", "name": "test anchor", "color": "red", "theta": -0.5},
        {"id": "1", "name": "test anchor 2", "theta": -0.9}
    ]
    data = [
        {"id": "0", "color": "orange", "weights": { "0": 0.5, "1": 0.5}},
        {"id": "1", "color": "orange", "weights": { "0": 0.0, "1": 0.0}},
        {"id": "2", "color": "orange", "weights": { "0": 1.0, "1": 1.1}},
        {"id": "3", "color": "blue", "weights": { "0": 1.0, "1": 0.0}, "labeled": True},
        {"id": "4", "color": "blue", "weights": { "0": 1.0, "1": 0.5}},
        {"id": "5", "color": "orange", "weights": { "0": 0.0, "1": 1.0}, "labeled": True},
        {"id": "6", "color": "blue", "weights": { "0": 0.0, "1": 0.5}},
    ]
    return anchors, data

In [ ]:
def simple_very_clustered_data():
    anchors = [
        {"id": "0", "name": "test anchor 2", "theta": -0.9}
    ]
    data = [{"id": str(i), "weights": { "0": float(i % 2) }} for i in range(20)]
    return anchors, data

# Basic Tests

## Sanity

* Does it run without errors?
* Can you click and drag the anchors?
* Can you hover over points and change selected data id?
* Can you click anchors to change selected anchor id?
* Do the table values update as you move anchors?
* Can you click the ring to add more anchors?

In [ ]:
a, d = simple_testing_data()
viz = av.AnchorViz(anchors=a, data=d, autoNorm = True, noiseMagnitude=0)
testing_dash(viz)

## Added noise

* Does setting noise magnitude spread out heavily clustered points?
* Is the noise magnitude evenly spread out around the _center_?

In [ ]:
a2, d2 = simple_very_clustered_data()
viz2 = av.AnchorViz(anchors=a2, data=d2, autoNorm=True, noiseMagnitude=0)
viz3 = av.AnchorViz(anchors=a2, data=d2, autoNorm=True, noiseMagnitude=30)
VBox([
    testing_dash(viz2),
    testing_dash(viz3)
])

---

In [ ]:
import random
def gen_big_data(n=100):
    points = []
    for i in range(n):
        point = dict(id=str(i), color=("blue" if i%2 == 0 else "orange"), labeled=(i%5 == 0), weights=dict())
        for key in ["0", "1", "2", "3"]:
            val = random.random()
            if key == "0" and i%2 == 0:
                val += .5
                val *= 4
            point["weights"][key] = val
        points.append(point)
    return points

In [ ]:
viz.data = gen_big_data()

In [ ]:
viz.modify_anchor(0, "name", "Blue feature")
viz.modify_anchor(1, "name", "Orange feature 1")
viz.modify_anchor(2, "name", "Orange feature 2")
viz.modify_anchor(3, "name", "Orange feature 3")

In [ ]:
alt_data = [
    {"id": "0", "color": "blue", "weights": { "0": 0.5, "1": 0.6}, "labeled": True},
    {"id": "1", "color": "blue", "weights": { "0": 0.0, "1": 0.1}, "labeled": True},
    {"id": "2", "color": "blue", "weights": { "0": 0.4, "1": 1.1}, "labeled": True},
    {"id": "3", "color": "blue", "weights": { "0": 0.0, "1": 1.0}, "labeled": True},
    {"id": "4", "color": "blue", "weights": { "0": 0.0, "1": 1.5}},
    {"id": "5", "color": "blue", "weights": { "0": 0.7, "1": 1.0}, "labeled": True},
    {"id": "6", "color": "blue", "weights": { "0": 0.5, "1": 0.5}},
    {"id": "7", "color": "blue", "weights": { "0": 0.4, "1": 0.7}},
]
viz.data = alt_data

In [ ]:
viz.modify_point(3, "color", "green")
viz.modify_point(4, "color", "green")

In [ ]:
viz.add_point({"color": "yellow", "weights": { "0": 0.6, "1": 0.2 }})

In [ ]:
viz.lassoedPointIDs = [1, 2]

In [ ]:
viz.selectedAnchorID = "1"

In [ ]:
viz.selectedDataPointID = "0"

In [ ]:
viz.anchors = [
    {"id": "0", "name": "things", "theta": 1.5},
    {"id": "1", "name": "things2", "theta": 2.5}
]

Add anchor via python

In [ ]:
# type(viz.anchors), type(viz.anchors[0])

In [ ]:
# viz.anchors = viz.anchors + [{"id": 5, "name": "things", "theta": 1.5}]

Modify anchor via python

In [ ]:
viz.modify_anchor(1, "color", "purple")

In [ ]:
anchor_to_change = viz.anchors[1]
anchor_to_change["color"] = "purple"
viz.anchors.pop(1)
viz.anchors = viz.anchors + [anchor_to_change]


In [ ]:
viz.anchors[1]

In [ ]:
viz.anchors[1] = dict(id=1, name="whaaaa", theta="0.5", color="orange")

In [ ]:
type(viz.anchors)

In [ ]:
viz.anchors[1]["color"] = "orange"

In [ ]:
viz.anchors[1]["color"] = "orange"
viz.notify_change({"name": "anchors", "type":"change"})

In [ ]:
anchors = viz.anchors
anchors[1]["color"] = "green"
viz.anchors = anchors

Delete anchor via python

In [ ]:
viz.anchors = [viz.anchors[0]]

In [ ]:
viz.anchors